#### 生成 Title View需要的数据
暂定群体是all，直接使用titles_all.csv数据


In [11]:
import pandas as pd
import json
from tqdm import tqdm
import random
import math
import numpy as np
from sklearn.cluster import KMeans

In [17]:
titles_all_df = pd.read_csv('题目数据特征/titles_all.csv')
st_df = pd.read_csv("人题数据/stu_title.csv")
titleInfo_df = pd.read_csv("../dataset/Data_TitleInfo.csv")
mergeData = pd.read_csv('../dataset/Data_MergedNew.csv')

sk_list = sorted(titleInfo_df['sub_knowledge'].unique())
k_list = sorted(titleInfo_df['knowledge'].unique())
t_list = sorted(titleInfo_df['title_ID'].unique())

In [42]:
def knowledge2TitleIDarr(knowledge):
    return titleInfo_df[titleInfo_df['knowledge']==knowledge]['title_ID'].unique().tolist()
# knowledge2TitleIDarr('r8S3g')
OutputArr = []
for k in k_list:
    Obj = {"knowledge": k, "titleArr": knowledge2TitleIDarr(k)}
    OutputArr.append(Obj)
print(OutputArr)

[{'knowledge': 'b3C9s', 'titleArr': ['Question_bumGRTJ0c8p4v5D6eHZa', 'Question_hZ5wXofebmTlzKB1jNcP', 'Question_FNg8X9v5zcbB1tQrxHR3']}, {'knowledge': 'g7R2j', 'titleArr': ['Question_oCjnFLbIs4Uxwek9rBpu', 'Question_5fgqjSBwTPG7KUV3it6O', 'Question_X3wF8QlTyi4mZkDp9Kae', 'Question_YWXHr4G6Cl7bEm9iF2kQ', 'Question_xqlJkmRaP0otZcX4fK3W']}, {'knowledge': 'k4W1c', 'titleArr': ['Question_lU2wvHSZq7m43xiVroBc']}, {'knowledge': 'm3D1v', 'titleArr': ['Question_h7pXNg80nJbw1C4kAPRm', 'Question_6RQj2gF3OeK5AmDvThUV', 'Question_4nHcauCQ0Y6Pm8DgKlLo', 'Question_TmKaGvfNoXYq4FZ2JrBu', 'Question_NixCn84GdK2tySa5rB1V', 'Question_n2BTxIGw1Mc3Zo6RLdUe', 'Question_7NJzCXUPcvQF4Mkfh9Wr', 'Question_ZTbD7mxr2OUp8Fz6iNjy', 'Question_Jr4Wz5jLqmN01KUwHa7g', 'Question_QRm48lXxzdP7Tn1WgNOf', 'Question_pVKXjZn0BkSwYcsa7C31', 'Question_oCjnFLbIs4Uxwek9rBpu']}, {'knowledge': 'r8S3g', 'titleArr': ['Question_VgKw8PjY1FR6cm2QI9XW', 'Question_q7OpB2zCMmW9wS8uNt3H', 'Question_fZrP3FJ4ebUogW9V7taS', 'Question_BW0ItEaym

In [37]:

TitleDataObjArr = []
for tid in t_list:
    titleDataObj = {}
    # 
    tdf = mergeData[mergeData['title_ID']==tid]
    tdf1 = titles_all_df[titles_all_df['title_ID']==tid]
    tdf2 = st_df[st_df['title_ID']==tid]
    tdf3 = titleInfo_df[titleInfo_df['title_ID']==tid]
    # 
    titleDataObj['title_ID'] = tid
    # 
    titleDataObj["belong_k"] = tdf3["knowledge"].tolist()[0]
    titleDataObj["belong_sk"] = tdf3["sub_knowledge"].tolist()[0]
    titleDataObj["t_Fullscore"] = tdf3['score'].tolist()[0]
    titleDataObj["t_mean_sbmnum"] = tdf1['t_mean_sbmnum'].tolist()[0]
    titleDataObj["t_mean_score"] = tdf1['t_mean_score'].tolist()[0]
    # 题目的所有提交时间
    titleDataObj["t_SubmitArr"] = tdf['time'].to_list()
    titleDataObj["t_SubmitNum"] = tdf.shape[0]
    # 得分频率
    valid_tdf = tdf[tdf['state'].isin(['Absolutely_Correct','Partially_Correct'])]
    titleDataObj["t_ValidSubmitNum"] = valid_tdf.shape[0]
    titleDataObj["t_score_distribute"] = valid_tdf['score'].value_counts().to_dict()
    titleDataObj["t_memArr"] = valid_tdf['memory'].tolist()
    titleDataObj["t_tcArr"] = valid_tdf['timeconsume'].tolist()
    # 
    TitleDataObjArr.append(titleDataObj)

json_str = json.dumps(TitleDataObjArr, ensure_ascii=False)
with open("TitleViewData2.json", "w", encoding="utf-8") as file:
    file.write(json_str)

找各个指标的最大最小值帮助前端比例尺映射

In [ ]:
min_pfm_tc = titles_all_df['t_min_pfm_tc'].min()
max_pfm_tc = titles_all_df['t_max_pfm_tc'].max()
print('时间复杂度范围',min_pfm_tc, max_pfm_tc)

min_pfm_mem = titles_all_df['t_min_pfm_mem'].min()
max_pfm_mem = titles_all_df['t_max_pfm_mem'].max()
print('空间复杂度范围',min_pfm_mem, max_pfm_mem)

min_mean_score = titles_all_df['t_mean_score'].min()
max_mean_score = titles_all_df['t_mean_score'].max()
print('平均分范围',min_mean_score, max_mean_score)
min_mean_sbmnum = titles_all_df['t_mean_sbmnum'].min()
max_mean_sbmnum = titles_all_df['t_mean_sbmnum'].max()
print('平均提交次数范围',min_mean_sbmnum, max_mean_sbmnum)

print('提交时间',mergeData['time'].min(), mergeData['time'].max())
valid_df = mergeData[mergeData['state'].isin(['Partially_Correct','Absolutely_Correct'])]
print('有效提交中')
print('最小mem',valid_df['memory'].min(),'最大mem', valid_df['memory'].max())
print('最小tc',valid_df['timeconsume'].min(),'最大tc', valid_df['timeconsume'].max())
# 真服了，就是又65536的mem，还是PC的提交

In [40]:
arr = mergeData['memory'].tolist()
x = np.array(arr).reshape(-1,1)
print(min(arr), max(arr))

km = KMeans(n_clusters=10, max_iter=1000, random_state=42).fit(x)
# print('@',km.cluster_centers_)
# print('@',km.labels_)
y = KMeans(n_clusters=10, max_iter=1000).fit_predict(x)
# print('@@@',y)

# 查看每个标签下的样本
for label in set(km.labels_):
    print(f"Cluster {label}:")
    indices = np.where(km.labels_ == label)[0]
    sample = x[indices]
    print(sample.min(),sample.max())



0 65536
Cluster 0:
396 4480
Cluster 1:
35948 44860
Cluster 2:
4540 11384
Cluster 3:
58424 65536
Cluster 4:
0 128
Cluster 5:
19936 27272
Cluster 6:
45288 54596
Cluster 7:
12456 19588
Cluster 8:
156 392
Cluster 9:
27536 34268


In [42]:
from sklearn.metrics import silhouette_score

# 使用 Silhouette Score 来确定最佳聚类数
silhouette_scores = []

# 遍历不同的聚类数
for k in tqdm(range(2, 11)):
    # kproto = KPrototypes(n_clusters=k, init='Huang', n_init=10, verbose=1)
    kmeans = KMeans(n_clusters=k, random_state=42)
    # clusters = kproto.fit_predict(data_for_clustering, categorical=[0, 1, 2, 3])
    clusters = kmeans.fit_predict(x)
    # 计算轮廓系数
    # numeric_part = data_for_clustering[:, 4:].astype(float)
    numeric_part = x.astype(float)
    score = silhouette_score(numeric_part, clusters)
    silhouette_scores.append(score)

import matplotlib.pyplot as plt
# 绘制 Silhouette Scores 图
plt.plot(range(2, 11), silhouette_scores, 'bx-')
plt.xlabel('聚类数')
plt.ylabel('轮廓系数')
plt.title('轮廓系数来确定最佳聚类数')
plt.show()

  0%|          | 0/9 [07:54<?, ?it/s]


KeyboardInterrupt: 